In [83]:
!pip install plotly 

    100% |████████████████████████████████| 37.3MB 741kB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 13.0MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 14.1MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 12.6MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 13.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/rute/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
mkl-random 1.0.1 requires cython, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
plotly.tools.set_credentials_file(username='rute.s.abreu', api_key='RZlLRB2Kzqcyo3xpzGjc')

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly

import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
col_labels = ['xmeas%02d_low' % x for x in [1,2,3,8,9,21]]+ ['xmeas%02d_high' % x for x in [1,2,3,8,9,21]] 
df_mat = np.zeros([12,12], dtype=int)

ref_df = pd.DataFrame(df_mat, columns = col_labels, index=col_labels)
ref_df.loc['xmeas01_low'][['xmeas21_high', 'xmeas09_low']] = [1,1]
ref_df.loc['xmeas09_low']['xmeas21_high'] = 1

ref_df.loc['xmeas01_high'][['xmeas21_low', 'xmeas09_high']] = [1,1]
ref_df.loc['xmeas09_high']['xmeas21_low'] = 1

In [3]:
def generate_conf_df(df, df_ref):
    confMatrix = np.zeros([2,2], dtype=int)
     
    for col, col_ref, j_index in zip(df.columns, df_ref.columns,  np.arange(0,len(df.columns))):
        for row, row_ref, index in zip(df[col], df_ref[col_ref], np.arange(0,len(df.columns))):
            if(j_index != index + df.columns.size/2 and j_index!=index and j_index != index - df.columns.size/2):
                if row == row_ref and row == 0:
                    confMatrix[1,1] += 1
                elif row == row_ref and row == 1:
                    confMatrix[0,0] += 1
                elif row != row_ref and row == 1:
                    confMatrix[0,1] += 1
                else:
                    confMatrix[1,0]+= 1 
    return pd.DataFrame(confMatrix,columns=["p", "n"], index = ["Y", "N"])

In [4]:
#roc fp*tp

def fp_rate(conf_df): #type 1 - error
    return conf_df["n"]["Y"]/(conf_df["n"]["Y"]+ conf_df["n"]["N"])

def tp_rate(conf_df): #since it is positive what are the odds of being classsified as positive - recall
    return conf_df['p']['Y']/(conf_df['p']['Y'] + conf_df['p']['N'])

def precision(conf_df): #from those i choose as correct, how many really were
    return conf_df['p']['Y']/(conf_df['p']['Y'] + conf_df['n']['Y'])

def fn_rate(conf_df): #type 2 error
    return (1 - fp_rate(conf_df))



In [5]:
def generate_df_valid_corrs(df, limit):
    df_valid = pd.DataFrame(np.zeros([df.shape[0], df.shape[1]], dtype=int), columns = df.columns, index= df.columns)
    for row in df.columns:
        for col in df.columns:
            if df[row][col] > limit:
                df_valid[row][col] = 1
                
    return df_valid 

In [103]:
t_levels = []

te_m5_h50 = pd.read_csv("resultados_defesa/te_m5_h50.csv", index_col=0)
t0 = np.mean(te_m5_h50.mean()) + 3* np.std(te_m5_h50.std())
t1 = np.mean(te_m5_h50.mean()) + 2* np.std(te_m5_h50.std())
t2 = np.mean(te_m5_h50.mean()) + np.std(te_m5_h50.std())

t_levels.append(t0)
t_levels.append(t1)
t_levels.append(t2)

conf_df = pd.DataFrame()
valid = pd.DataFrame()
fpr = [] 
tpr = []

levels = list(te_m5_h50.values.flatten())
levels.sort(reverse=True)
level_rates = []

for level in levels :
    valid = generate_df_valid_corrs(te_m5_h50, level)
    conf_df = generate_conf_df(valid, ref_df)
    fpr.append(fp_rate(conf_df))
    tpr.append(tp_rate(conf_df))
    level_rates.append([fp_rate(conf_df),tp_rate(conf_df), level])

trace = go.Scatter(x = fpr, y = tpr)

py.iplot([trace])
 

In [105]:
level_chsd = 0.01816308533330481

In [107]:
level_rates


[[0.0, 0.0, 0.3987931046471108],
 [0.008771929824561403, 0.0, 0.15256422399391975],
 [0.017543859649122806, 0.0, 0.13618717255543394],
 [0.02631578947368421, 0.0, 0.13387294151521045],
 [0.03508771929824561, 0.0, 0.12462660239906605],
 [0.043859649122807015, 0.0, 0.09862151654220302],
 [0.05263157894736842, 0.0, 0.08214614395417215],
 [0.05263157894736842, 0.16666666666666666, 0.07659895402281522],
 [0.05263157894736842, 0.3333333333333333, 0.07571203515670545],
 [0.06140350877192982, 0.3333333333333333, 0.07272906859108677],
 [0.06140350877192982, 0.5, 0.055674922579944816],
 [0.07017543859649122, 0.5, 0.03191854266853026],
 [0.07894736842105263, 0.5, 0.02258806941918814],
 [0.08771929824561403, 0.5, 0.02037067258143916],
 [0.08771929824561403, 0.6666666666666666, 0.018317309064842968],
 [0.08771929824561403, 0.8333333333333334, 0.01816308533330481],
 [0.09649122807017543, 0.8333333333333334, 0.01768949825545004],
 [0.10526315789473684, 0.8333333333333334, 0.016401934273716987],
 [0.1

In [150]:
t_roc = 0.01816308533330481
t_roc2 = 0.055674922579944816
t_roc_df = generate_df_valid_corrs(te_m5_h50,t_roc) 
t_roc2_df = generate_df_valid_corrs(te_m5_h50,t_roc2)

In [151]:
conf_roc = generate_conf_df(t_roc_df, ref_df)
conf_roc2 = generate_conf_df(t_roc2_df, ref_df)

In [152]:
print('tp: ', tp_rate(conf_roc))
print('fp: ', fp_rate(conf_roc))

tp:  0.8333333333333334
fp:  0.08771929824561403


In [153]:
print('tp: ', tp_rate(conf_roc2))
print('fp: ', fp_rate(conf_roc2))

tp:  0.5
fp:  0.06140350877192982
